In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
clmns = [
    "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15","Open 16","Close 16","Open 17","Close 17","Open 18","Close 18","Open 19","Close 19","Open 20","Close 20","Open 21","Close 21","Open 22","Close 22","Open 23","Close 23","Open 24","Close 24","Open 25","Close 25","Open 26","Close 26","Open 27","Close 27","Open 28","Close 28","Open 29","Close 29","Open 30","Close 30",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-3,3))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(30, data.shape[0]):
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        sugg = "sell"
        if data[i][3] > data[i-1][3]:
            sugg = "buy"
        arr = np.array(gggrow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [3]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files[:100]:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [4]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"730d","1h")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [ ]:
folder_name = extract_data(500)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

In [4]:
xTrain,xTest,yTrain,yTest = spliting(pd.concat([pd.read_parquet("/content/drive/MyDrive/chc_data1.parquet"),pd.read_parquet("/content/drive/MyDrive/chc_data2.parquet")]))

(6079338, 60) (6079338, 2)
(675482, 60) (675482, 2)


In [5]:
model = load_model("CHC_G115.h5")

In [6]:
model = Sequential()

model.add(Dense(3000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))
model.add(Dense(1500, activation='relu'))


opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3000)              183000    
                                                                 
 dense_1 (Dense)             (None, 2000)              6002000   
                                                                 
 dense_2 (Dense)             (None, 1500)              3001500   
                                                                 
 dense_3 (Dense)             (None, 1500)              2251500   
                                                                 
 dense_4 (Dense)             (None, 1500)              2251500   
                                                                 
 dense_5 (Dense)             (None, 2)                 3002      
                                                                 
Total params: 13,692,502
Trainable params: 13,692,502
No

In [ ]:
model.fit(xTrain,yTrain,epochs=10,batch_size=25000,validation_data=(xTest,yTest))

Epoch 1/10
244/244 [==============================] - 316s 1s/step - loss: 0.0982 - accuracy: 0.9707 - val_loss: 0.0889 - val_accuracy: 0.9745
Epoch 2/10
244/244 [==============================] - 304s 1s/step - loss: 0.0622 - accuracy: 0.9821 - val_loss: 0.0947 - val_accuracy: 0.9720
Epoch 3/10
244/244 [==============================] - 303s 1s/step - loss: 0.0487 - accuracy: 0.9853 - val_loss: 0.1079 - val_accuracy: 0.9683
Epoch 4/10
197/244 [=======================>......] - ETA: 56s - loss: 0.0439 - accuracy: 0.9863

In [8]:
model.save("CHC_G115.h5")

In [ ]:
yf.download("btc-usd",period="max",interval="1mo")

In [ ]:
def process_for_prediction(data):
        i = -1
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten().reshape(-1,1)
        return arr
def make_prediction(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [ ]:
make_prediction("btc-usd","max","1mo")

[*********************100%***********************]  1 of 1 completed


array([[0.99922645, 0.00252116]], dtype=float32)

In [ ]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten()
        return arr
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_monthly,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [ ]:
make_prediction("s","x")